Portfolio Project

For my portfolio project, I have selected to use the legendary Titanic dataset from Kaggle. 
https://www.kaggle.com/competitions/titanic

The model will be a random forest classifier that will predict which passengers survived the Titanic shipwreck. The Titanic shipwreck occurred in 1912 on her maiden voyage and resulted in the death of 1502 of the 2224 passengers and crew on board. There was some element of luck in who survived the disaster, however certain groups of people were more likely to survive than others. 

For my portfolio project I will build a predictive model to answer the question, 'what sorts of people were more likely to survive?' using passenger data from the Kaggle dataset.


First load in the data from the Kaggle download
And import any packages needed

In [89]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

train.head()
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


Some feature engineering
- Set M or F to 0 and 1
- Remove NANs
- Change Embarked to numeric


In [90]:
train['Sex'] = train['Sex'].map({'male': 0, 'female': 1})
train['Embarked'].fillna('S', inplace=True)
train['Embarked'] = train['Embarked'].map({"S": 0, "C": 1, "Q": 2})
# Replace the NA values in age and fare with the median value
train['Age'].fillna(train['Age'].median(), inplace=True) 
train['Fare'].fillna(train['Fare'].median(), inplace=True)

train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,1
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,0
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,0


Select features to use for training the model. Features deemed important are
- Pclass
- Sex
- Age
- Sibsp
- Parch
- Fare
- Embarked

The goal feature is whether or not the passenger survived

In [91]:
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
X = train[features]
y = train['Survived']

Now build the random forest classifier and train using the training data

In [92]:
rf = RandomForestClassifier(random_state=42)


# Perform the search
rf.fit(X, y)

test['Sex'] = test['Sex'].map({'male': 0, 'female': 1})
test['Embarked'].fillna('S', inplace=True)
test['Embarked'] = test['Embarked'].map({"S": 0, "C": 1, "Q": 2})
# Replace the NA values in age and fare with the median value
test['Age'].fillna(test['Age'].median(), inplace=True) 
test['Fare'].fillna(test['Fare'].median(), inplace=True)

test.head()

X_test = test[features]
print(X_test)
# # y_test = test['Survived']

y_predict = rf.predict(X_test)

print(y_predict)

     Pclass  Sex   Age  SibSp  Parch      Fare  Embarked
0         3    0  34.5      0      0    7.8292         2
1         3    1  47.0      1      0    7.0000         0
2         2    0  62.0      0      0    9.6875         2
3         3    0  27.0      0      0    8.6625         0
4         3    1  22.0      1      1   12.2875         0
..      ...  ...   ...    ...    ...       ...       ...
413       3    0  27.0      0      0    8.0500         0
414       1    1  39.0      0      0  108.9000         1
415       3    0  38.5      0      0    7.2500         0
416       3    0  27.0      0      0    8.0500         0
417       3    0  27.0      1      1   22.3583         1

[418 rows x 7 columns]
[0 0 0 1 0 0 0 0 1 0 0 0 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 0 0 0 1 0 1 1 1
 0 0 1 0 1 0 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 0 0
 1 0 0 1 0 1 1 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0
 1 1 0 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 

Now lets test how the model performs on the training data

In [95]:
pred = rf.predict(X)

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


accuracy = accuracy_score(y, pred)
cm = confusion_matrix(y, pred)

tn, fp, fn, tp = cm.ravel()
specificity = tn / (tn + fp)

print("Confusion Matrix:\n", cm)
print("\nAccuracy: ", accuracy)
print("Specificity: ", specificity)
print("\nClassification Report:\n", classification_report(y, pred))


Confusion Matrix:
 [[545   4]
 [ 14 328]]

Accuracy:  0.9797979797979798
Specificity:  0.9927140255009107

Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.99      0.98       549
           1       0.99      0.96      0.97       342

    accuracy                           0.98       891
   macro avg       0.98      0.98      0.98       891
weighted avg       0.98      0.98      0.98       891



It performs very well on the training data